<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/XLM_Roberta_Fine_tuning_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

In [ ]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install transformers datasets
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

Found existing installation: tensorflow 2.8.2+zzzcolab20220929150707
Uninstalling tensorflow-2.8.2+zzzcolab20220929150707:
  Successfully uninstalled tensorflow-2.8.2+zzzcolab20220929150707
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 26.6 MB/s 
     |████████████████████████████████| 432 kB 56.1 MB/s 
     |████████████████████████████████| 163 kB 68.7 MB/s 
     |████████████████████████████████| 7.6 MB 56.9 MB/s 
     |████████████████████████████████| 212 kB 74.2 MB/s 
     |████████████████████████████████| 115 kB 62.1 MB/s 
     |████████████████████████████████| 127 kB 75.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

In [ ]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"
!awk NF < dump.txt > kant.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1739k  100 1739k    0     0  1081k      0  0:00:01  0:00:01 --:--:-- 1081k


In [ ]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

#tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")
#model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

#tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
#model = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")


Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
!shuf -n1000 kant.txt > kant_teste.txt

In [ ]:
from datasets import load_dataset
datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-af6ecd1f73183fc1/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (612 > 512). Running this sequence through the model will result in indexing errors


#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (680 > 512). Running this sequence through the model will result in indexing errors


#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

exemplo de entrada

In [ ]:
tokenized_datasets["train"][4]

{'input_ids': [0, 3145, 11549, 9153, 8, 91234, 2],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#block_size = tokenizer.model_max_length
block_size = 32


In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
#@title Step 13: Pre-training the Model
%%time
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 15332
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 9590


Step,Training Loss
500,2.898200
1000,2.683700
1500,2.562000
2000,2.477200
2500,2.396100
3000,2.342500
3500,2.275300
4000,2.247200
4500,2.199900
5000,2.129500




Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 51min 28s, sys: 7min 1s, total: 58min 29s
Wall time: 58min 25s


TrainOutput(global_step=9590, training_loss=2.210643718190437, metrics={'train_runtime': 3505.2141, 'train_samples_per_second': 43.741, 'train_steps_per_second': 2.736, 'total_flos': 2528621047987200.0, 'train_loss': 2.210643718190437, 'epoch': 10.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1995
  Batch size = 8


Perplexity: 6.27


In [ ]:
eval_results

{'eval_loss': 1.8364803791046143,
 'eval_runtime': 12.2173,
 'eval_samples_per_second': 163.293,
 'eval_steps_per_second': 20.463,
 'epoch': 10.0}

In [ ]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
trainer.save_model("./KantaiBERT")

Saving model checkpoint to ./KantaiBERT
Configuration saved in ./KantaiBERT/config.json
Model weights saved in ./KantaiBERT/pytorch_model.bin


In [ ]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./KantaiBERT",
    tokenizer=tokenizer
)

loading configuration file ./KantaiBERT/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.23.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading configuration file ./KantaiBERT/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "./KantaiBERT",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  

In [ ]:
fill_mask("O rapaz olhou para o <mask> ")

[{'score': 0.22369031608104706,
  'token': 177122,
  'token_str': 'chão',
  'sequence': 'O rapaz olhou para o chão'},
 {'score': 0.09750895202159882,
  'token': 48682,
  'token_str': 'médico',
  'sequence': 'O rapaz olhou para o médico'},
 {'score': 0.0936368927359581,
  'token': 118427,
  'token_str': 'marido',
  'sequence': 'O rapaz olhou para o marido'},
 {'score': 0.05697939917445183,
  'token': 175194,
  'token_str': 'céu',
  'sequence': 'O rapaz olhou para o céu'},
 {'score': 0.03810923546552658,
  'token': 13343,
  'token_str': 'outro',
  'sequence': 'O rapaz olhou para o outro'}]

In [ ]:
fill_mask("A moça olhou para o <mask> ")

[{'score': 0.1730472445487976,
  'token': 118427,
  'token_str': 'marido',
  'sequence': 'A moça olhou para o marido'},
 {'score': 0.13274118304252625,
  'token': 48682,
  'token_str': 'médico',
  'sequence': 'A moça olhou para o médico'},
 {'score': 0.09116166830062866,
  'token': 175194,
  'token_str': 'céu',
  'sequence': 'A moça olhou para o céu'},
 {'score': 0.08979421854019165,
  'token': 177122,
  'token_str': 'chão',
  'sequence': 'A moça olhou para o chão'},
 {'score': 0.03531690686941147,
  'token': 187397,
  'token_str': 'menino',
  'sequence': 'A moça olhou para o menino'}]

In [ ]:
fill_mask("Comprou uma <mask> na loja. ")

[{'score': 0.31681978702545166,
  'token': 20533,
  'token_str': 'carta',
  'sequence': 'Comprou uma carta na loja.'},
 {'score': 0.24454960227012634,
  'token': 185350,
  'token_str': 'cadeira',
  'sequence': 'Comprou uma cadeira na loja.'},
 {'score': 0.12325749546289444,
  'token': 2349,
  'token_str': 'casa',
  'sequence': 'Comprou uma casa na loja.'},
 {'score': 0.019163521006703377,
  'token': 30092,
  'token_str': 'coisa',
  'sequence': 'Comprou uma coisa na loja.'},
 {'score': 0.01597200706601143,
  'token': 101591,
  'token_str': 'peça',
  'sequence': 'Comprou uma peça na loja.'}]

In [ ]:
fill_mask("A mulher não é <mask>. ")

[{'score': 0.10641437768936157,
  'token': 91276,
  'token_str': 'bonita',
  'sequence': 'A mulher não é bonita.'},
 {'score': 0.1056104227900505,
  'token': 44277,
  'token_str': 'homem',
  'sequence': 'A mulher não é homem.'},
 {'score': 0.07729542255401611,
  'token': 12479,
  'token_str': 'assim',
  'sequence': 'A mulher não é assim.'},
 {'score': 0.07539550215005875,
  'token': 52636,
  'token_str': 'mulher',
  'sequence': 'A mulher não é mulher.'},
 {'score': 0.04109249636530876,
  'token': 57873,
  'token_str': 'mãe',
  'sequence': 'A mulher não é mãe.'}]

In [ ]:
fill_mask("O homem não é <mask>. ")

[{'score': 0.3443267345428467,
  'token': 44277,
  'token_str': 'homem',
  'sequence': 'O homem não é homem.'},
 {'score': 0.06193983927369118,
  'token': 44053,
  'token_str': 'padre',
  'sequence': 'O homem não é padre.'},
 {'score': 0.0317402184009552,
  'token': 166899,
  'token_str': 'pecado',
  'sequence': 'O homem não é pecado.'},
 {'score': 0.022063270211219788,
  'token': 8634,
  'token_str': 'bom',
  'sequence': 'O homem não é bom.'},
 {'score': 0.01950421743094921,
  'token': 135162,
  'token_str': 'perfeito',
  'sequence': 'O homem não é perfeito.'}]

In [ ]:
fill_mask("Ele é um bom <mask>. ")

[{'score': 0.3703443109989166,
  'token': 44277,
  'token_str': 'homem',
  'sequence': 'Ele é um bom homem.'},
 {'score': 0.30951687693595886,
  'token': 114789,
  'token_str': 'rapaz',
  'sequence': 'Ele é um bom rapaz.'},
 {'score': 0.10795821249485016,
  'token': 38260,
  'token_str': 'amigo',
  'sequence': 'Ele é um bom amigo.'},
 {'score': 0.016941042616963387,
  'token': 84983,
  'token_str': 'coração',
  'sequence': 'Ele é um bom coração.'},
 {'score': 0.016340838745236397,
  'token': 187397,
  'token_str': 'menino',
  'sequence': 'Ele é um bom menino.'}]

In [ ]:
fill_mask("Ela é uma boa <mask>. ")

[{'score': 0.23675066232681274,
  'token': 179879,
  'token_str': 'criatura',
  'sequence': 'Ela é uma boa criatura.'},
 {'score': 0.19339397549629211,
  'token': 52636,
  'token_str': 'mulher',
  'sequence': 'Ela é uma boa mulher.'},
 {'score': 0.1320735365152359,
  'token': 55207,
  'token_str': 'amiga',
  'sequence': 'Ela é uma boa amiga.'},
 {'score': 0.05175231769680977,
  'token': 22068,
  'token_str': 'alma',
  'sequence': 'Ela é uma boa alma.'},
 {'score': 0.04246633127331734,
  'token': 57873,
  'token_str': 'mãe',
  'sequence': 'Ela é uma boa mãe.'}]

In [ ]:
fill_mask("Faz de conta que ainda é <mask>. ")

[{'score': 0.09034914523363113,
  'token': 15234,
  'token_str': 'tarde',
  'sequence': 'Faz de conta que ainda é tarde.'},
 {'score': 0.06952247023582458,
  'token': 11721,
  'token_str': 'pouco',
  'sequence': 'Faz de conta que ainda é pouco.'},
 {'score': 0.06760646402835846,
  'token': 79426,
  'token_str': 'criança',
  'sequence': 'Faz de conta que ainda é criança.'},
 {'score': 0.04443119838833809,
  'token': 8634,
  'token_str': 'bom',
  'sequence': 'Faz de conta que ainda é bom.'},
 {'score': 0.029098480939865112,
  'token': 96872,
  'token_str': 'jovem',
  'sequence': 'Faz de conta que ainda é jovem.'}]

In [ ]:
fill_mask("Depois da tempestade vem a <mask>. ")

[{'score': 0.07353661209344864,
  'token': 2436,
  'token_str': 'vida',
  'sequence': 'Depois da tempestade vem a vida.'},
 {'score': 0.0431353896856308,
  'token': 23291,
  'token_str': 'noite',
  'sequence': 'Depois da tempestade vem a noite.'},
 {'score': 0.0418689027428627,
  'token': 27761,
  'token_str': 'lua',
  'sequence': 'Depois da tempestade vem a lua.'},
 {'score': 0.039513710886240005,
  'token': 31522,
  'token_str': 'verdade',
  'sequence': 'Depois da tempestade vem a verdade.'},
 {'score': 0.03670697659254074,
  'token': 119433,
  'token_str': 'alegria',
  'sequence': 'Depois da tempestade vem a alegria.'}]

In [ ]:
fill_mask("Mais vale um pássaro na mão do que <mask> voando. ")

[{'score': 0.2400769144296646,
  'token': 286,
  'token_str': 'um',
  'sequence': 'Mais vale um pássaro na mão do que um voando.'},
 {'score': 0.12609295547008514,
  'token': 193,
  'token_str': 'ir',
  'sequence': 'Mais vale um pássaro na mão do que ir voando.'},
 {'score': 0.10057839751243591,
  'token': 13343,
  'token_str': 'outro',
  'sequence': 'Mais vale um pássaro na mão do que outro voando.'},
 {'score': 0.09620093554258347,
  'token': 788,
  'token_str': 'uma',
  'sequence': 'Mais vale um pássaro na mão do que uma voando.'},
 {'score': 0.08542916178703308,
  'token': 25629,
  'token_str': 'ficar',
  'sequence': 'Mais vale um pássaro na mão do que ficar voando.'}]

In [ ]:
fill_mask("Não tinha <mask>, mas almoçou mesmo assim. ")

[{'score': 0.13539738953113556,
  'token': 5799,
  'token_str': 'nada',
  'sequence': 'Não tinha nada, mas almoçou mesmo assim.'},
 {'score': 0.08850663155317307,
  'token': 204382,
  'token_str': 'almoço',
  'sequence': 'Não tinha almoço, mas almoçou mesmo assim.'},
 {'score': 0.07357217371463776,
  'token': 26216,
  'token_str': 'café',
  'sequence': 'Não tinha café, mas almoçou mesmo assim.'},
 {'score': 0.06951604783535004,
  'token': 99145,
  'token_str': 'riso',
  'sequence': 'Não tinha riso, mas almoçou mesmo assim.'},
 {'score': 0.05365394055843353,
  'token': 95907,
  'token_str': 'leite',
  'sequence': 'Não tinha leite, mas almoçou mesmo assim.'}]

In [ ]:
fill_mask("Bebeu um copo d'água, pois tinha <mask>. ")

[{'score': 0.07275236397981644,
  'token': 123299,
  'token_str': 'medo',
  'sequence': "Bebeu um copo d'água, pois tinha medo."},
 {'score': 0.038881123065948486,
  'token': 11721,
  'token_str': 'pouco',
  'sequence': "Bebeu um copo d'água, pois tinha pouco."},
 {'score': 0.0366661511361599,
  'token': 99145,
  'token_str': 'riso',
  'sequence': "Bebeu um copo d'água, pois tinha riso."},
 {'score': 0.035144660621881485,
  'token': 3628,
  'token_str': 'muito',
  'sequence': "Bebeu um copo d'água, pois tinha muito."},
 {'score': 0.030086176469922066,
  'token': 235013,
  'token_str': 'lágrimas',
  'sequence': "Bebeu um copo d'água, pois tinha lágrimas."}]

In [ ]:
fill_mask("Sem saber, ele descobriu um <mask>. ")

[{'score': 0.32733097672462463,
  'token': 165801,
  'token_str': 'segredo',
  'sequence': 'Sem saber, ele descobriu um segredo.'},
 {'score': 0.0840831845998764,
  'token': 137558,
  'token_str': 'secreto',
  'sequence': 'Sem saber, ele descobriu um secreto.'},
 {'score': 0.04991142079234123,
  'token': 879,
  'token_str': 'dia',
  'sequence': 'Sem saber, ele descobriu um dia.'},
 {'score': 0.029515186324715614,
  'token': 26018,
  'token_str': 'livro',
  'sequence': 'Sem saber, ele descobriu um livro.'},
 {'score': 0.018334437161684036,
  'token': 95757,
  'token_str': 'sonho',
  'sequence': 'Sem saber, ele descobriu um sonho.'}]

In [ ]:
fill_mask("Pedro fez fortuna vendendo <mask>. ")

[{'score': 0.125064417719841,
  'token': 27,
  'token_str': '...',
  'sequence': 'Pedro fez fortuna vendendo....'},
 {'score': 0.06702392548322678,
  'token': 5,
  'token_str': '.',
  'sequence': 'Pedro fez fortuna vendendo..'},
 {'score': 0.05795770511031151,
  'token': 10964,
  'token_str': 'tudo',
  'sequence': 'Pedro fez fortuna vendendo tudo.'},
 {'score': 0.05409489944577217,
  'token': 73295,
  'token_str': 'livros',
  'sequence': 'Pedro fez fortuna vendendo livros.'},
 {'score': 0.04353716969490051,
  'token': 51881,
  'token_str': 'dinheiro',
  'sequence': 'Pedro fez fortuna vendendo dinheiro.'}]